In [1]:
import os
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=10'

In [2]:
import numpy as np
import jax
import treams.special as ts
import treams

In [3]:
x = y = np.linspace(-500, 500, 121)
X, Y = np.meshgrid(x,y)
X = X.flatten()
Y = Y.flatten()
Z = np.zeros_like(X)
pos = np.array([
  [-350, 350, -300, 300, -300, 300],
  [0,    0,   -165, -165, 165, 165],
  [0,    0,   0,    0,    0,   0]
]).T

In [4]:
k0 = 0.011180689785085548-0.0004307384124945234j
basis = treams.CylindricalWaveBasis.default(0, 3, 6, positions=pos)
basis = basis[basis.pol==0]
material = treams.Material(1)

In [5]:
ks = material.ks(k0)[basis.pol]
krhos = material.krhos(k0, basis.kz, basis.pol) *(1-0.2j)

In [6]:
%%timeit 
rcyl = ts.car2cyl(np.array([X,Y,Z]).T[:, np.newaxis] - basis.positions)

11.1 ms ± 555 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
rcyl = ts.car2cyl(np.array([X,Y,Z]).T[:, np.newaxis] - basis.positions)

In [8]:
rcyl[..., basis.pidx, 0].shape

(14641, 42)

In [9]:
42*14641

614922

In [10]:
%timeit ts.vcw_M(0,basis.m, krhos * rcyl[..., basis.pidx, 0], rcyl[..., basis.pidx, 1], rcyl[..., basis.pidx, 2])

3.15 s ± 90.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
from scipy.special import hankel1

In [16]:
x = np.linspace(1, 1000, int(1e6))

In [17]:
%timeit hankel1(1, x)

339 ms ± 14.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit np.exp(x)

<magic-timeit>:1: RuntimeWarning: overflow encountered in exp


8.07 ms ± 84.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
